In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
import os
from pathlib import Path
import math

home = Path.home() / 'group' / 'project'

# load the scores

In [ ]:
list((home / 'results' / 'matched_filter' / 'sparse_grid').iterdir())

In [ ]:
result_file = '/storage/home/adz6/group/project/results/matched_filter/sparse_grid/220211_radial_pitch_angle_matched_filter_score_grid.npz'

In [ ]:
np_file = np.load(result_file)

In [ ]:
for i, key in enumerate(np_file.keys()): print(key)

In [ ]:
scores = np_file['scores']
r_grid = np_file['r']
pa_grid = np_file['pa']

In [ ]:
angle = 87.0
radius = 0.01
inds = np.argwhere(np.logical_and(r_grid == radius, pa_grid == angle)).squeeze()

score = scores[inds[0], inds[1]]

# define the survival functions

In [ ]:
x = np.linspace(0, 12, 201)

noise_sf = stats.rayleigh.sf(x, loc=0, scale=(1/math.sqrt(2)))
signal_sf = stats.rice.sf(x, score, loc=score*(1 - 1 / math.sqrt(2)), scale=1/math.sqrt(2))

In [ ]:
match = 1

signal_sf = stats.rice.sf(
    x.reshape((1, x.size)),
    match * scores.flatten().reshape(scores.size, 1),
    loc = match * scores.flatten().reshape(scores.size, 1) * (1 - 1 / math.sqrt(2)),
    scale=1/math.sqrt(2)
)

In [ ]:
signal_sf.shape

# plot for match = 1

In [ ]:
sns.set_theme(context='poster')
fig = plt.figure(figsize=(13, 8))
ax = fig.add_subplot(1,1,1)


ax.plot(noise_sf, signal_sf,)
#ax.plot(noise_sf, select_signal_sf.mean(axis=0),  label=r'$\theta\in85-89$ deg and $r\in1-4$ cm')

ax.set_ylabel('True Positive Rate (Efficiency)')
ax.set_xlabel('False Positive Rate (Background)')
ax.set_xscale('log')
ax.set_xlim(1e-10, 1)
#ax.legend(loc=4)

plt.tight_layout()
name = '220214_ideal_mf_roc_curves_selected_parameters'
save_path = home/'plots'/'analysis'/'matched_filter'/'dense_grid'/'score_grid'
save_path.mkdir(parents=True, exist_ok=True)

#plt.savefig(str(save_path / name))

# plot for range of match

In [ ]:
sns.set_theme(context='poster')
fig = plt.figure(figsize=(13, 8))
ax = fig.add_subplot(1,1,1)

x = np.linspace(0, 12, 201)
noise_sf = stats.rayleigh.sf(x, loc=0, scale=(1/math.sqrt(2)))

for match in [0.1, 0.3, 0.5, 0.7, 0.9, 1.0]:
    
    signal_sf = stats.rice.sf(x, match * score, loc=match*score*(1 - 1 / math.sqrt(2)), scale=1/math.sqrt(2))


    ax.plot(noise_sf, signal_sf,)
#ax.plot(noise_sf, select_signal_sf.mean(axis=0),  label=r'$\theta\in85-89$ deg and $r\in1-4$ cm')

ax.set_ylabel('True Positive Rate (Efficiency)')
ax.set_xlabel('False Positive Rate (Background)')
ax.set_xscale('log')
ax.set_xlim(1e-10, 1)
#ax.legend(loc=4)

plt.tight_layout()
name = '220214_ideal_mf_roc_curves_selected_parameters'
#save_path = home/'plots'/'analysis'/'matched_filter'/'dense_grid'/'score_grid'
#save_path.mkdir(parents=True, exist_ok=True)

# compare range of match againsts dnn

In [ ]:
path = Path.home()/'group'/'project'/'results'/'machine_learning'/'dnn'/'triggering'/'eval_outputs'/'dense_grid'/'220228_pitch87.0_rad1cm_fft'/'pdf_and_cdf.npz'

file = np.load(path)
for i,key in enumerate(file):
    print(key)

In [ ]:
sns.set_theme(context='poster')
fig = plt.figure(figsize=(13, 8))
ax = fig.add_subplot(1,1,1)

x = np.linspace(0, 12, 201)
noise_sf = stats.rayleigh.sf(x, loc=0, scale=(1/math.sqrt(2)))
match_list = np.flip([0.5, 0.7,1.0])
cmap = sns.color_palette('mako_r', n_colors=len(match_list))
clist = sns.color_palette('deep')

for i, match in enumerate(match_list):
    
    signal_sf = stats.rice.sf(x, match * score, loc=match*score*(1 - 1 / math.sqrt(2)), scale=1/math.sqrt(2))

    ax.plot(noise_sf, signal_sf,color=cmap[i], label=f'Match = {match}')
ax.plot(1-file['cdf_noise'], 1-file['cdf_signal'],color=clist[3], label='DNN')

ax.set_ylabel('True Positive Rate (Efficiency)')
ax.set_xlabel('False Positive Rate (Background)')
ax.set_xscale('log')
ax.set_xlim(1e-6, 1)
ax.legend(loc=4)

plt.tight_layout()
name = '220214_ideal_mf_roc_curves_selected_parameters'
#save_path = home/'plots'/'analysis'/'matched_filter'/'dense_grid'/'score_grid'
#save_path.mkdir(parents=True, exist_ok=True)

In [ ]:
r_min = 0.00
r_max = 0.05
pa_min = 84.0
pa_max = 90.0

In [ ]:
pa_grid.shape

In [ ]:
r_select = np.logical_and(
    r_grid.flatten() >= r_min,
    r_grid.flatten() <= r_max
)

pa_select = np.logical_and(
    pa_grid.flatten() >= pa_min,
    pa_grid.flatten() <= pa_max
)

select = np.logical_and(
    r_select,
    pa_select
)

In [ ]:
select_signal_sf = signal_sf[select]

In [ ]:
sns.set_theme(context='poster')
fig = plt.figure(figsize=(13, 8))
ax = fig.add_subplot(1,1,1)


ax.plot(noise_sf, signal_sf.mean(axis=0), label=r'$\theta\in84-90$ deg and $r\in0-5$ cm' )
ax.plot(noise_sf, select_signal_sf.mean(axis=0),  label=r'$\theta\in85-89$ deg and $r\in1-4$ cm')

ax.set_ylabel('True Positive Rate (Efficiency)')
ax.set_xlabel('False Positive Rate (Background)')
ax.set_xscale('log')
ax.set_xlim(1e-10, 1)
ax.legend(loc=4)

plt.tight_layout()
name = '220214_ideal_mf_roc_curves_selected_parameters'
save_path = home/'plots'/'analysis'/'matched_filter'/'dense_grid'/'score_grid'
save_path.mkdir(parents=True, exist_ok=True)

#plt.savefig(str(save_path / name))

# visualize ROC for match assumptions

In [ ]:
sns.set_theme(context='talk')
fig = plt.figure(figsize=(13, 8))
ax = fig.add_subplot(1,1,1)

match_ratios = np.array([1.0, 0.9, 0.75, 0.5, 0.3, 0.1])
tpr_list = []

for match in match_ratios:
    signal_sf = stats.rice.sf(
        x.reshape((1, x.size)),
        match * scores.flatten().reshape(scores.size, 1),
        loc = match * scores.flatten().reshape(scores.size, 1) * (1 - 1 / math.sqrt(2)),
        scale=1/math.sqrt(2)
    )
    
    ax.plot(noise_sf, signal_sf[select].mean(axis=0), label=f'Match = {match}')
    
    tpr_list.append(signal_sf[select].mean(axis=0))
    
tpr_array = np.array(tpr_list)
fpr_array = noise_sf

ax.set_ylabel('True Positive Rate (Efficiency)')
ax.set_xlabel('False Positive Rate (Background)')
ax.set_xscale('log')
ax.set_xlim(1e-10, 1)
ax.legend(loc=2)
#ax.plot(np.linspace(0,1, 10), np.linspace(0,1, 10), color='grey', linestyle='--')


plt.tight_layout()
name = '220215_mf_roc_curves_estimates_match_efficiency_reduction'
save_path = home/'plots'/'analysis'/'matched_filter'/'dense_grid'/'score_grid'
save_path.mkdir(parents=True, exist_ok=True)

#plt.savefig(str(save_path / name))

# save roc curves

In [ ]:
save_path = Path.home()/'group'/'project'/'results'/'matched_filter'/'roc_curve'/'220223_matched_filter_roc_curves_diff_match_angles84to90_rad0to5.npz'


In [ ]:
np.savez(save_path, match=match_ratios, fpr=fpr_array, tpr=tpr_array)